In [1]:
%pylab inline
import xarray as xr
import pandas as pd

attrs = {
   'airpres':  {'units': 'Pa', 'long_name': 'Air pressure'},
   'airtemp':  {'units': 'K', 'long_name': 'Air temperature'},
   'spechum':  {'units': 'g g-1', 'long_name': 'Specific humidity'},
   'windspd':  {'units': 'Wind speed', 'long_name': 'm s-1'},
   'SWRadAtm': {'units': 'W m-2', 'long_name': 'Downward shortwave radiation'},
   'LWRadAtm': {'units': 'W m-2', 'long_name': 'Downward longwave radiation'},
   'pptrate':  {'units': 'kg m-2 s-1', 'long_name': 'Precipitation rate'}}

mpl.rcParams['figure.figsize'] = (18, 10)

Populating the interactive namespace from numpy and matplotlib


In [2]:
def spechum(rh, T, p):
    T0 = 273.15
    return rh * np.exp((17.67*(T-T0))/(T-29.65)) / (0.263*p)

In [3]:
raw_p = pd.read_csv('../data/MHP2011to2017_SnowDepth_SWE_Precip.csv')
bounds = pd.DatetimeIndex(
    pd.date_range('2010/10/01', '2010/12/31', freq='H'), name='time')[1:-1]
raw_p.index = pd.DatetimeIndex(raw_p['DateTime'], name='time')

In [4]:
raw = pd.read_csv('../data/cues2011to2017_Radiation_Albedo_WindSpeed'
                  '_GroundTemp_AirTemp_RH_AirPressure_SnowDepth_SWE.csv')
raw.index = pd.DatetimeIndex(raw['DateTime'], name='time')
# This has one less data step than raw_p
raw = raw[:-1]

In [5]:
lats = [45.30]
lons = [5.77]
elev = 1325.
bounds = raw_p.index

shape = (len(bounds), 1, )
dims = ('time', 'hru', )
coords = {'time': bounds}

met_data = xr.Dataset(coords=coords)
met_data.time.encoding['calendar'] = 'standard'
met_data.time.encoding['units'] = 'hours since 1990-01-01'
summa_vars = ['airpres', 'airtemp', 'spechum', 
              'windspd', 'SWRadAtm', 'LWRadAtm', 'pptrate']
for varname in summa_vars:
    met_data[varname] = xr.DataArray(data=np.full(shape, np.nan),
                                     coords=coords, dims=dims,
                                     name=varname, attrs=attrs[varname])

In [6]:
met_data['airpres' ].loc[{'hru': 0}] = raw['AirPressure_mb'] * 100
met_data['airtemp' ].loc[{'hru': 0}] = raw['AirTemp_C'] + 273.15
met_data['windspd' ].loc[{'hru': 0}] = raw['wind_speed_meters_per_sec']
met_data['SWRadAtm'].loc[{'hru': 0}] = raw['uplooking_broadband_watts_per_meterxx2']
met_data['LWRadAtm'].loc[{'hru': 0}] = raw['uplooking_longwave_watts_per_meterxx2']
met_data['pptrate' ].loc[{'hru': 0}] = raw_p['precip_cm_hr'] * 10 / 3600
met_data['spechum' ].loc[{'hru': 0}] = spechum(raw['RH'] * 100, 
                                               raw['AirTemp_C'] + 273.15, 
                                               raw['AirPressure_mb'] * 100)

In [7]:
ds_local_attrs = xr.open_dataset('./summa_setup_template/params/local_attributes.nc')
ds_template = xr.open_dataset('./CUES/cues_2016_summa_setup/forcings/cues2016.nc')
met_data['hruId'] = ds_local_attrs['hruId']
met_data['latitude'] = ds_local_attrs['latitude']
met_data['longitude'] = ds_local_attrs['longitude']
met_data['data_step'] = [3600.]

FileNotFoundError: [Errno 2] No such file or directory: b'/pool0/data/andrbenn/workspace/whw2019_snow_modeling/notebooks/summa_setup_template/params/local_attributes.nc'

In [ ]:
met_data.to_netcdf('../data/cues_2011-2017.nc')
met_data